In [ ]:
import os
from typing import Any, Literal, TypedDict

from flashrank import Ranker
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import (
    DocumentCompressorPipeline,
    EmbeddingsFilter,
    LLMChainExtractor,
)
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_compressors import FlashrankRerank
from langchain_community.document_loaders import TextLoader
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_semantic_cache import ensure_semantic_cache
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph
from pydantic import BaseModel, Field, ValidationError

from utils import display_markdown


class LearningModule(TypedDict):
    query: str


class IntentClassifier(BaseModel):
    intent: Literal["Learning", "Progress Check", "Help Request", "Goal Setting"] = (
        Field(
            description="The user's primary intent, chosen from one of the following: 'Learning', 'Progress Check', 'Help Request', or 'Goal Setting'."
        )
    )
    confidence: float = Field(
        description="A confidence score (between 0.0 and 1.0) indicating how certain the model is about the classified intent."
    )

In [ ]:
ensure_semantic_cache()

2025-07-22 22:01:36,010 - INFO - Use pytorch device_name: cpu
2025-07-22 22:01:36,013 - INFO - Load pretrained SentenceTransformer: thenlper/gte-base
2025-07-22 22:01:45,720 - INFO - Semantic cache initialized and set globally.


SemanticCache(database_path='.langchain.db', faiss_index_path='semantic_faiss', similarity_threshold=0.4, max_cache_size=1000, memory_cache_size=100, batch_size=10, enable_quantization=False, embedding_cache=OrderedDict(), memory_cache={}, metrics={'cache_hits': 0, 'cache_misses': 0, 'semantic_hits': 0, 'embedding_time': 0, 'search_time': 0, 'memory_hits': 0}, executor=<concurrent.futures.thread.ThreadPoolExecutor object at 0x7fdbd4123390>, lock=<unlocked _thread.RLock object owner=0 count=0 at 0x7fdbf9b4d700>, embeddings=HuggingFaceEmbeddings(model_name='thenlper/gte-base', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), vector_store=None, _lazy_loaded=False)

In [ ]:
llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    temperature=0.7,
)

llm_intent = llm.with_structured_output(IntentClassifier)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """# Intent Classification Task

Classify the user query into exactly ONE category:
- **Learning**: Seeking knowledge, tutorials, explanations
- **Progress Check**: Reviewing status, tracking advancement
- **Help Request**: Troubleshooting, problem-solving assistance
- **Goal Setting**: Planning objectives, defining targets

## Output Format:
Category: [CATEGORY]
Confidence: [0.0-1.0]

## Examples:
Query: "How do I learn Python?"
Category: Learning
Confidence: 0.95

Query: "Am I on track with my fitness goals?"
Category: Progress Check
Confidence: 0.90

## User Query:
{query}"""
)

chain = prompt | llm_intent

In [ ]:
chain.invoke(
    "What are the recommended practices for effective machine learning workflows?"
)  # type: ignore

2025-07-22 22:01:47,068 - INFO - Loading faiss.


2025-07-22 22:01:47,241 - INFO - Successfully loaded faiss.
2025-07-22 22:01:47,271 - INFO - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
2025-07-22 22:01:47,280 - INFO - Loaded FAISS index from semantic_faiss
2025-07-22 22:01:49,855 - INFO - Semantic match found with score 0.0119


IntentClassifier(intent='Goal Setting', confidence=0.85)

# The Learning query preprocessing
## Knowledge base creation

In [ ]:
def document_processor(file_path: str) -> list[Document]:
    if not os.path.exists(file_path):
        print("The file path provided does not exist. Please check it again and retry")
        return None

    loader = TextLoader(file_path)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(docs)

    return docs

In [ ]:
docs = document_processor("romeo.txt")

In [ ]:
def create_store(docs: list[Document], persist_dir: str, embeddings):
    if os.path.exists(persist_dir):
        print(f"Loaded the vector store from {persist_dir}")
        store = FAISS.load_local(
            persist_dir, embeddings, allow_dangerous_deserialization=True
        )

    else:
        if not docs:
            print("No documents were passed")
            return None

        if not os.path.exists(persist_dir):
            os.makedirs(persist_dir)
            print(f"Successfuly created the persist directory at {persist_dir}")

        print(f"Creating the new vector store with {len(docs)} documents")
        store = None

        for i in range(0, len(docs), 50):
            batch = docs[i : i + 50]

            print(
                f"Processing batch {i//50 + 1}/{(len(docs) + 49)//50} ({len(batch)} documents)"
            )
            if store is None:
                store = FAISS.from_documents(batch, embeddings)
            else:
                store.add_documents(batch)

        store.save_local(persist_dir)
        print(f"Vector store created and saved at {persist_dir}")

    return store

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-base", model_kwargs={"local_files_only": True}
)

2025-07-22 22:01:50,009 - INFO - Use pytorch device_name: cpu
2025-07-22 22:01:50,013 - INFO - Load pretrained SentenceTransformer: thenlper/gte-base


In [ ]:
store = create_store(docs, "faiss_store", embeddings)

Loaded the vector store from faiss_store


In [ ]:
ranker = Ranker(
    model_name="ms-marco-MiniLM-L-12-v2",
    cache_dir=os.path.expanduser("~/.cache/flashrank"),
)

In [ ]:
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 10
retriever = store.as_retriever(search_kwargs={"k": 10})  # type: ignore
retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [ ]:
compressor_pipeline = DocumentCompressorPipeline(
    transformers=[
        EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.6),
        EmbeddingsRedundantFilter(embeddings=embeddings, similarity_threshold=0.95),
        FlashrankRerank(client=ranker),
        LLMChainExtractor.from_llm(llm),
    ]
)

In [ ]:
# compressor = FlashrankRerank(client=ranker)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor_pipeline, base_retriever=retriever
)

In [ ]:
class LearningState(TypedDict):
    query: str
    qa_result: str
    relevant_docs: list[Document]
    learning_response: str
    user_profile: dict[str, Any]
    final_explanation: str
    learning_level: str
    learning_style: str
    difficulty_adjustments: str
    personalized_content: str
    learning_activities: list[str]
    related_topics: list[str]
    fallback_used: bool
    search_method: Literal["knowledge_base", "web_search"]

In [ ]:
tavily_search = TavilySearch(
    max_results=1,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False,
)

In [ ]:
class MockDoc:
    def __init__(self, content, url):
        self.page_content = content
        self.metadata = {"source": url}

In [ ]:
def knowledge_base_search(state: LearningState):
    try:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
        )
        answer = qa_chain.invoke(state["query"])

        # LLM quality check
        if _is_adequate(answer["result"], state["query"]):
            return {
                **state,
                "qa_result": answer["result"],
                "relevant_docs": answer["source_documents"],
                "context_chunks": [
                    doc.page_content for doc in answer["source_documents"]
                ],
                "search_method": "knowledge_base",
                "fallback_used": False,
            }

    except Exception as e:
        print(f"Knowledge base search failed: {str(e)}")

    # Web search fallback
    try:
        web_results = tavily_search.invoke(
            f"Romeo and Juliet by Shakespeare: {state['query']}"
        )
        mock_docs = [
            MockDoc(res["content"], res["url"]) for res in web_results["results"]
        ]
        return {
            **state,
            "qa_result": web_results["answer"],
            "relevant_docs": mock_docs,
            "context_chunks": [res["content"] for res in web_results["results"]],
            "search_method": "web_search",
            "fallback_used": True,
        }
    except Exception as e:
        print(f"Web search failed: {str(e)}")

    return {
        **state,
        "qa_result": "I couldn't find relevant information about this topic.",
        "relevant_docs": [],
        "context_chunks": [],
        "search_method": "none",
        "fallback_used": True,
    }


def _is_adequate(answer: str, query: str) -> bool:
    """LLM judge for answer quality."""
    prompt = f"""Question: {query}
    Answer: {answer}

    Is this answer adequate for the Romeo and Juliet question? Consider if it's specific, relevant, and informative.
    Respond only: YES or NO"""

    try:
        # structured_llm = llm.with_structured_output
        response = llm.invoke(prompt)
        return "YES" in response.content.upper()
    except Exception as e:
        print(f"An Error occurred {str(e)}")
        return len(answer) > 50 and "don't know" not in answer.lower()


def generate_learning_response(state: LearningState):
    if not state.get("relevant_docs"):
        return {
            **state,
            "learning_response": "I couldn't find specific information about this topic in Romeo and Juliet. Please try rephrasing your question or ask about a different aspect of the play.",
        }

    context_text = "\n".join(
        [
            f"Context {i+1}:\n{doc.page_content}"
            for i, doc in enumerate(state.get("relevant_docs", []))
        ]
    )

    prompt_template = """You are a literature tutor for Romeo and Juliet.

    Initial Answer: {qa_result}
    Context: {context}
    User Query: {query}

    Provide an enhanced educational response with quotes, themes, and analysis.

    Enhanced Learning Response:"""

    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | StrOutputParser()

    learning_response = chain.invoke(
        {
            "qa_result": state["qa_result"],
            "context": context_text,
            "query": state["query"],
        }
    )

    return {**state, "learning_response": learning_response}

In [ ]:
class PersonalizedEngine(BaseModel):
    personalized_content: str = Field(
        description="Adjusted complexity to match the learner's level with a maximum of 300 words."
    )
    learning_activities: list[str] = Field(
        description="A list of at most 5 tailored tasks or exercises to reinforce understanding."
    )
    related_topics: list[str] = Field(
        description="At most 5 topics closely connected to the main content for deeper exploration."
    )
    difficulty_adjustments: str = Field(
        description="Explanation of how the content difficulty was modified."
    )


class UserProfile(BaseModel):
    learning_level: str = "intermediate"
    learning_style: str = "visual"
    interests: list[str] = ["character_analysis", "themes"]
    previous_topics: list[str] = []


def content_personalization_engine(state: dict):
    # Error handling for user profile data
    profile_data = state.get("user_profile", {})
    try:
        profile = UserProfile(**profile_data)
    except ValidationError as e:
        print(f"Invalid profile data: {e}")
        profile = UserProfile()  # Use defaults

    if "learning_response" not in state:
        raise ValueError("learning_response is required in state")
    if "query" not in state:
        raise ValueError("query is required in state")

    learning_response = state["learning_response"]
    if len(learning_response) > 2000:
        learning_response = learning_response[:2000] + "..."

    personalization_prompt = ChatPromptTemplate.from_template(
        """Personalize this Romeo and Juliet content for a {learning_level} learner who prefers {learning_style} learning.

        Original content: {learning_response}

        KEEP ALL RESPONSES SHORT:
        - personalized_content: Rewrite for {learning_level} level (max 300 words)
        - learning_activities: List at most 5 activities (one sentence each)
        - related_topics: List at most 5 topic names only
        - difficulty_adjustments: What you changed (max 50 words)
        """
    )

    llm_structured = llm.with_structured_output(PersonalizedEngine)
    chain = personalization_prompt | llm_structured

    personalization_content = chain.invoke(
        {
            "learning_level": profile.learning_level,
            "learning_style": profile.learning_style,
            "interests": ", ".join(profile.interests),
            "previous_topics": ", ".join(profile.previous_topics),
            "learning_response": state["learning_response"],
            "query": state["query"],
        }
    )

    # Extract structured fields from the response
    return {
        **state,
        "personalized_content": personalization_content.personalized_content,  # type: ignore
        "learning_activities": personalization_content.learning_activities,  # type: ignore
        "related_topics": personalization_content.related_topics,  # type: ignore
        "difficulty_adjustments": personalization_content.difficulty_adjustments,  # type: ignore
        "learning_level": profile.learning_level,
        "learning_style": profile.learning_style,
    }

In [ ]:
def generate_explanation(state: LearningState):
    explanation_prompt = ChatPromptTemplate.from_template(
        """Answer this Romeo and Juliet question for a {learning_level} student:

        Question: {query}

        Content: {personalized_content}

        Activities: {learning_activities}

        Related topics: {related_topics}

        Keep your response under 300 words and make it engaging for a {learning_level} learner.
        """
    )

    chain = explanation_prompt | llm
    final_explanation = chain.invoke(
        {
            "query": state["query"],
            "learning_level": state.get("learning_level", "intermediate"),
            "learning_style": state.get("learning_style", "visual"),
            "personalized_content": state.get("personalized_content", ""),
            "learning_activities": "\n".join(state.get("learning_activities", [])),
            "related_topics": ", ".join(state.get("related_topics", [])),
            "difficulty_adjustments": state.get("difficulty_adjustments", ""),
        }
    )

    return {
        **state,
        "final_explanation": (
            final_explanation.content
            if hasattr(final_explanation, "content")
            else str(final_explanation)
        ),
    }

In [ ]:
# conn = sqlite3.connect("memory.db", check_same_thread=False)
# memory = SqliteSaver(conn)
# config = {"configurable": {"thread_id": "1"}}


workflow = StateGraph(LearningState)

# Add nodes
workflow.add_node("knowledge_base_search", knowledge_base_search)
workflow.add_node("generate_learning_response", generate_learning_response)
workflow.add_node("content_personalization_engine", content_personalization_engine)
workflow.add_node("generate_explanation", generate_explanation)

# Define edges

workflow.add_edge("knowledge_base_search", "generate_learning_response")
workflow.add_edge("generate_learning_response", "content_personalization_engine")
workflow.add_edge("content_personalization_engine", "generate_explanation")
workflow.set_entry_point("knowledge_base_search")

app = workflow.compile()  # Remove memory checkpointing

In [ ]:
result = app.invoke(
    {"query": "What are the key events that lead to Romeo and Juliet's deaths?"},
    # config=config,
)

2025-07-22 22:12:19,954 - INFO - Memory cache hit
2025-07-22 22:12:19,957 - INFO - Memory cache hit
2025-07-22 22:12:19,961 - INFO - Memory cache hit
2025-07-22 22:12:19,964 - INFO - Memory cache hit
2025-07-22 22:12:19,967 - INFO - Memory cache hit
2025-07-22 22:12:19,974 - INFO - Memory cache hit
2025-07-22 22:12:19,981 - INFO - Memory cache hit
2025-07-22 22:12:19,991 - INFO - Memory cache hit


In [ ]:
%load_ext autoreload
%autoreload 2
display_markdown(result, title="Romeo and Juliet Analysis Results")

# 🎭 Romeo and Juliet Analysis Results

🎭 **Source:** Romeo & Juliet Knowledge Base

---

## ❓ Question

> **❝ What are the key events that lead to Romeo and Juliet's deaths? ❞**

## 💡 Qa Result

The key events that lead to Romeo and Juliet's deaths include:

1. **The Feud**: The longstanding feud between the Montagues and Capulets sets the stage for tragedy.

2. **Secret Marriage**: Romeo and Juliet fall in love and secretly marry, which complicates their situation.

3. **Tybalt's Death**: After Tybalt kills Mercutio, Romeo avenges his friend’s death by killing Tybalt, which leads to Romeo's banishment from Verona.

4. **Juliet's Despair**: Juliet is devastated by Tybalt's death and Romeo's banishment. To avoid marrying Paris, she seeks help from Friar Laurence.

5. **The Plan**: Friar Laurence gives Juliet a potion that will make her appear dead for 42 hours, with the intention of reuniting her with Romeo.

6. **Miscommunication**: The message about Juliet's faked death fails to reach Romeo. Instead, he learns of her "death" through misinformation.

7. **Romeo's Suicide**: Believing Juliet to be truly dead, Romeo buys poison and takes it in Juliet's tomb.

8. **Juliet's Awakening**: Juliet wakes up shortly after Romeo's death, finds him dead beside her, and then takes her own life with his dagger.

These events, driven by passion, fate, and miscommunication, culminate in the
tragic deaths of both lovers.

## 📖 Learning Response

Certainly! The tragic deaths of Romeo and Juliet are the culmination of a series
of fateful events, miscommunications, and intense emotions that reflect the
themes of love, fate, and the consequences of familial conflict. Here’s an
enhanced analysis of the key events leading to their tragic end, supported by
quotes and thematic insights:

### 1. The Feud
The longstanding feud between the Montagues and Capulets establishes the tragic
backdrop of the play. This animosity creates an environment where love is
forbidden. The Prince of Verona articulates the consequences of this feud when
he declares, “If ever you disturb our streets again, Your lives shall pay the
forfeit of the peace” (Act 1, Scene 1). This sets in motion the events that lead
to chaos and tragedy.

### 2. Secret Marriage
Romeo and Juliet’s love blossoms in secrecy, leading them to marry in haste.
Friar Laurence, who hopes their union will mend the rift between their families,
warns, “Wisely and slow. They stumble that run fast” (Act 2, Scene 3). Their
impulsive decision to marry without their families’ knowledge foreshadows the
impending disaster.

### 3. Tybalt’s Death
The death of Tybalt marks a turning point in the play. After Tybalt kills
Mercutio, Romeo avenges his friend’s death, proclaiming, “I am fortune’s fool”
(Act 3, Scene 1). This moment underscores the theme of fate; Romeo feels trapped
by the circumstances surrounding him, which leads to his banishment.

### 4. Juliet’s Despair
With Romeo banished and Tybalt dead, Juliet is left in despair. She states, “O
serpent heart, hid with a flowering face!” (Act 3, Scene 2), expressing her
conflicted feelings about Romeo. Her desperation drives her to seek Friar
Laurence’s help to avoid marrying Paris, showcasing her determination and the
societal pressures she faces.

### 5. The Plan
Friar Laurence devises a plan to help Juliet escape her predicament. He gives
her a potion that will make her appear dead for 42 hours. He explains, “Take
thou this vial... and in this borrowed likeness of shrunk death,” (Act 4, Scene
1). This plan, while well-intentioned, is fraught with risks and sets the stage
for the tragic miscommunication.

### 6. Miscommunication
The failure of the message to reach Romeo is a critical moment in the narrative.
Instead of learning of Juliet’s faked death, he hears only of her “death” from
Balthasar, who tells him, “Her body sleeps in Capels’ monument” (Act 5, Scene
1). This miscommunication underscores the theme of fate and the tragic
consequences of chance.

### 7. Romeo’s Suicide
Believing Juliet to be truly dead, Romeo arrives at her tomb, where he laments,
“Here’s to my love!” before taking poison (Act 5, Scene 3). His despair
illustrates the intensity of his love and the devastating impact of the feud
that has led to this moment of irreversible loss.

### 8. Juliet’s Awakening
When Juliet awakens to find Romeo dead beside her, she is devastated. She cries
out, “O happy dagger! This is thy sheath!” (Act 5, Scene 3), choosing to take
her own life. This final act of love highlights the depth of their bond but also
the tragic consequences of their families' enmity.

### Themes and Analysis
The themes of love, fate, and the impact of family loyalty are woven throughout
these events. The passionate love between Romeo and Juliet is portrayed as both
beautiful and destructive, ultimately leading to their demise. The concept of
fate is prevalent, as the characters frequently reference the stars and destiny,
suggesting that their tragic end was predestined. Additionally, the destructive
nature of the family feud illustrates how societal constructs can deeply affect
personal relationships, leading to tragic outcomes.

In conclusion, the intertwined events that lead to the deaths of Romeo and
Juliet are a powerful commentary on love, fate, and the consequences of familial
conflict. Each event builds upon the last, creating a tragic inevitability that
resonates with audiences and exemplifies Shakespeare’s mastery of narrative and
character development.

## 🎯 Final Explanation

Romeo and Juliet is a powerful story about love and tragedy, filled with
misunderstandings and family conflicts. Here are the key events that lead to the
heartbreaking deaths of the two young lovers:

1. **The Feud**: The bitter rivalry between their families, the Montagues and Capulets, makes their love forbidden. The Prince warns that further fighting will have dire consequences (Act 1, Scene 1).

2. **Secret Marriage**: In hopes of uniting their families, Romeo and Juliet marry in secret. Friar Laurence advises them to take their time, saying, "Wisely and slow" (Act 2, Scene 3).

3. **Tybalt’s Death**: After Tybalt kills Mercutio, Romeo seeks revenge and kills Tybalt. He realizes the consequences of his actions when he says, "I am fortune’s fool" (Act 3, Scene 1), leading to his banishment.

4. **Juliet’s Despair**: Juliet is heartbroken when Romeo is exiled. She expresses her pain, feeling betrayed by fate (Act 3, Scene 2).

5. **The Plan**: Friar Laurence gives Juliet a potion to fake her death, hoping to reunite her with Romeo (Act 4, Scene 1).

6. **Miscommunication**: Romeo receives news of Juliet's "death" but doesn’t know it’s a ruse, leading to his sense of hopelessness (Act 5, Scene 1).

7. **Romeo’s Suicide**: Believing Juliet is truly dead, Romeo drinks poison to join her in death (Act 5, Scene 3).

8. **Juliet’s Awakening**: When Juliet awakens and finds Romeo dead, she chooses to die alongside him, declaring, "O happy dagger!" (Act 5, Scene 3).

These events show how deep love can lead to tragic outcomes, especially when
mixed with feuds and fate.

For your activities, consider creating a storyboard or character map to
visualize these events, or draw a scene that captures the theme of fate. Have
fun exploring this timeless tale!

---

## 📚 Additional Information

### 📋 Difficulty Adjustments

Simplified language and structure were used to clarify the main events and
themes, making it more accessible for intermediate learners.

### ✅ Learning Activities

✅ Create a storyboard illustrating the key events leading to the deaths of Romeo and Juliet.

✅ Design a character map showing relationships and conflicts between characters.

✅ Draw a scene from the play that represents the theme of fate.

✅ Make a visual timeline of events in the play using images and quotes.

✅ Create a poster that shows the main themes of love and conflict in Romeo and Juliet.

### 📚 Learning Level

intermediate

### 🎨 Learning Style

visual

### 📋 Personalized Content

Romeo and Juliet's tragic deaths happen because of misunderstandings, strong
emotions, and the feud between their families. Here are the key events that lead
to their sad ending:

1. **The Feud**: The long-standing fight between the Montagues and Capulets makes their love forbidden. The Prince warns, "If you disturb our streets again, your lives will pay the price" (Act 1, Scene 1).

2. **Secret Marriage**: Romeo and Juliet secretly marry, hoping to bring peace. Friar Laurence advises, "Wisely and slow. They stumble that run fast" (Act 2, Scene 3).

3. **Tybalt’s Death**: After Tybalt kills Mercutio, Romeo seeks revenge, stating, "I am fortune’s fool" (Act 3, Scene 1). This leads to his banishment.

4. **Juliet’s Despair**: With Romeo gone, Juliet cries, "O serpent heart, hid with a flowering face!" (Act 3, Scene 2), showing her pain and confusion.

5. **The Plan**: Friar Laurence gives Juliet a potion to fake her death, saying, "Take thou this vial..." (Act 4, Scene 1).

6. **Miscommunication**: Romeo learns of Juliet’s death but doesn’t know it’s fake, leading to his despair (Act 5, Scene 1).

7. **Romeo’s Suicide**: Believing Juliet is dead, he drinks poison (Act 5, Scene 3).

8. **Juliet’s Awakening**: Finding Romeo dead, Juliet chooses to die with him, saying, "O happy dagger! This is thy sheath!" (Act 5, Scene 3).

The themes of love and fate run throughout their story, showing how family
conflicts can lead to tragic outcomes.

### 🔗 Related Topics

🎭 Shakespearean Tragedy

🎭 Themes of Love in Literature

🎭 The Role of Fate in Storytelling

🎭 Character Analysis of Romeo and Juliet

🎭 Family Feuds in Literature

### 📜 Relevant Docs

### 📜 Text Reference 1
---
📖 **Source:** romeo.txt

> *"FIRST WATCH.
Sovereign, here lies the County Paris slain,
And Romeo dead, and Juliet, dead before,
Warm and new kill’d.

PRINCE.
Search, seek, and know how this foul murder comes.

CAPULET.
O heaven! O wife, look how our daughter bleeds!
This dagger hath mista’en, for lo, his house
Is empty on the b..."*

### 📜 Text Reference 2
---
📖 **Source:** romeo.txt

> *""I married them; and their stol’n marriage day Was Tybalt’s doomsday, whose untimely death Banish’d the new-made bridegroom from this city; For whom, and not for Tybalt, Juliet pin’d.""*

### 📜 Text Reference 3
---
📖 **Source:** romeo.txt

> *"NURSE.
Tybalt is gone, and Romeo banished,
Romeo that kill’d him, he is banished.

JULIET.
O God! Did Romeo’s hand shed Tybalt’s blood?"*

# Goal setting

In [ ]:
class AgentState(TypedDict):
    query: str
    goal: str
    smart_goal: str
    curriculum: list[str]
    learning_path: list[str]
    requirements: list[str]

In [ ]:
GOAL_SETTING_TEMPLATE = """You are an educational assistant.
Create a clear, basic learning goal from the user's input.

User Input: {query}"""

SMART_GOAL_TEMPLATE = """Transform this goal into SMART format with specific metrics:
- Specific: Exactly what will be accomplished?
- Measurable: How will you track progress? (e.g., "complete 5 projects", "score 80% on exam")
- Achievable: Realistic timeframe and scope
- Relevant: Why this matters for the user's objectives
- Time-bound: Specific deadline (e.g., "within 3 months")

Goal: {goal}"""

LEARNING_PATH_TEMPLATE = """Create a sequential learning path to achieving this SMART Goal
                        Break it down into logical, progressive steps that build upon each other.
                        SMART GOAL: {smart_goal}
                        """

CURRICULUM_TEMPLATE = """Design a curriculum structure aligned with this learning path.
Create modules that support the learning objectives and path.

Learning Path: {learning_path}
SMART Goal: {smart_goal}
"""

PREREQUISITES_TEMPLATE = """Identify the prerequisites needed before starting this curriculum.
Consider knowledge, skills, tools, or resources required.

Curriculum: {curriculum}
SMART Goal: {smart_goal}
"""

In [ ]:
class Goal(BaseModel):
    goal: str = Field(description="A structured well formatted goal to be achieved")


class SmartGoal(BaseModel):
    smart_goal: str = Field(description="A SMART formatted goal with specific timeline")


class LearningPath(BaseModel):
    learning_path: list[str] = Field(
        description="Sequential learning steps to achieve the goal"
    )


class Curriculum(BaseModel):
    modules: list[str] = Field(
        description="Curriculum modules aligned with learning objectives"
    )


class Prerequisites(BaseModel):
    requirements: list[str] = Field(
        description="Prerequisites required before starting on the goal"
    )

In [ ]:
def create_structured_node(
    template: str,
    output_model: BaseModel,
    input_key,
    output_key: str,
    model_attribute: str = None,
):
    """Factory function to create structured LLM nodes with consistent pattern"""

    def node_function(state: AgentState):
        prompt = ChatPromptTemplate.from_template(template)
        structured_llm = llm.with_structured_output(output_model)
        chain = prompt | structured_llm

        # Handle single input
        if isinstance(input_key, str):
            input_data = {input_key: state[input_key]}
        # Handle multiple inputs
        else:
            input_data = {key: state[key] for key in input_key}

        result = chain.invoke(input_data)

        # Extract the output - use model_attribute if specified, otherwise use output_key
        attribute_name = model_attribute or output_key
        output_value = getattr(result, attribute_name)
        return {**state, output_key: output_value}

    return node_function

In [ ]:
goal_setting_node = create_structured_node(
    template=GOAL_SETTING_TEMPLATE,
    output_model=Goal,
    input_key="query",
    output_key="goal",
)

smart_goal_node = create_structured_node(
    template=SMART_GOAL_TEMPLATE,
    output_model=SmartGoal,
    input_key="goal",
    output_key="smart_goal",
)

learning_path_node = create_structured_node(
    template=LEARNING_PATH_TEMPLATE,
    output_model=LearningPath,
    input_key="smart_goal",
    output_key="learning_path",
)

curriculum_node = create_structured_node(
    template=CURRICULUM_TEMPLATE,
    output_model=Curriculum,
    input_key=["learning_path", "smart_goal"],
    output_key="curriculum",
    model_attribute="modules",  # Get result.modules but store as "curriculum"
)

prerequisite_checker_node = create_structured_node(
    template=PREREQUISITES_TEMPLATE,
    output_model=Prerequisites,
    input_key=["curriculum", "smart_goal"],
    output_key="requirements",
)

In [ ]:
# Create the workflow
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("goal_setting_node", goal_setting_node)
workflow.add_node("smart_goal_node", smart_goal_node)
workflow.add_node("learning_path_node", learning_path_node)
workflow.add_node("curriculum_node", curriculum_node)
workflow.add_node("prerequisite_checker_node", prerequisite_checker_node)

# Add edges to define the flow
workflow.set_entry_point("goal_setting_node")
workflow.add_edge("goal_setting_node", "smart_goal_node")
workflow.add_edge("smart_goal_node", "learning_path_node")
workflow.add_edge("learning_path_node", "curriculum_node")
workflow.add_edge("curriculum_node", "prerequisite_checker_node")
workflow.set_finish_point("prerequisite_checker_node")

# Compile the graph
graph = workflow.compile()

In [ ]:
initial_state = {
    "query": "I need to learn machine learning in 6 months",
    "goal": "",
    "smart_goal": "",
    "curriculum": [],
    "learning_path": [],
    "requirements": [],
}

# Run the graph
result = graph.invoke(initial_state)

2025-07-22 22:02:22,779 - INFO - SQLite cache hit
2025-07-22 22:02:22,811 - INFO - SQLite cache hit
2025-07-22 22:02:22,829 - INFO - SQLite cache hit
2025-07-22 22:02:22,852 - INFO - SQLite cache hit
2025-07-22 22:02:22,869 - INFO - SQLite cache hit


In [ ]:
%load_ext autoreload
%autoreload 2
display_markdown(result, title="Goals Analysis")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 🎭 Goals Analysis

## ❓ Question

> **❝ I need to learn machine learning in 6 months ❞**

---

## 📚 Additional Information

### 📋 Curriculum

## 📋 Module 1: Course Selection and Planning  
   - Researching reputable online machine learning courses  
   - Evaluating course content for hands-on projects  
   - Selecting a course and creating a learning schedule  
   - Setting up a study environment and resources

## 📋 Module 2: Fundamental Concepts of Machine Learning  
   - Introduction to Machine Learning  
   - Supervised vs. Unsupervised Learning  
   - Data Preprocessing Techniques  
   - Evaluation Metrics for Machine Learning Models  
   - Hands-on practice with exercises and quizzes

## 📋 Module 3: Programming Foundations for Machine Learning  
   - Introduction to Python/R for Machine Learning  
   - Overview of Libraries: Scikit-learn, TensorFlow, Keras  
   - Coding exercises focusing on library functionalities  
   - Implementing basic data manipulations and visualizations

## 📋 Module 4: Introduction to Basic Machine Learning Algorithms  
   - Understanding Linear Regression and its implementation  
   - Exploring Decision Trees and their applications  
   - Hands-on coding projects using sample datasets  
   - Setting weekly milestones for algorithm implementations

## 📋 Module 5: Collaboration and Community Engagement  
   - Joining online forums and study groups  
   - Participating in discussions and Q&A sessions  
   - Sharing progress and challenges with peers  
   - Learning from collaborative experiences

## 📋 Module 6: Real-World Dataset Implementation  
   - Selecting a dataset for a basic machine learning model  
   - Documenting the data analysis process  
   - Implementing the chosen model and analyzing results  
   - Tracking progress through project milestones

## 📋 Module 7: Advanced Machine Learning Models  
   - Introduction to Random Forests  
   - Understanding Support Vector Machines  
   - Implementing advanced models on various datasets  
   - Evaluating and comparing model performance

## 📋 Module 8: Course Assessments and Feedback  
   - Completing all course assessments with a target score of 80%  
   - Reviewing feedback on assignments and quizzes  
   - Identifying areas for improvement  
   - Setting goals for continuous learning

## 📋 Module 9: Final Project Preparation  
   - Selecting a dataset of interest for the final project  
   - Defining the problem statement and project scope  
   - Outlining the approach and models to be used  
   - Preparing a project timeline and milestones

## 📋 Module 10: Conducting the Final Project  
   - Implementing at least three machine learning models  
   - Analyzing results and drawing insights  
   - Preparing a presentation for the final project  
   - Practicing presentation skills and receiving peer feedback

## 📋 Module 11: Review and Revisions  
   - Reviewing the final project based on feedback  
   - Making necessary revisions to the project  
   - Finalizing the presentation and project report  
   - Reflecting on the learning journey and future applications

### 📋 Goal

To acquire a foundational understanding of machine learning concepts,
algorithms, and practical applications within 6 months, enabling the ability to
implement basic machine learning models and analyze data effectively.

### 📋 Learning Path

- Research and select a reputable online machine learning course that covers fundamental concepts and algorithms, ensuring it allows for hands-on projects.

- Create a learning schedule to allocate specific times each week for studying course material and completing assignments.

- Complete introductory modules of the course to grasp fundamental concepts such as supervised vs. unsupervised learning, data preprocessing, and evaluation metrics.

- Practice coding skills in Python or R, focusing on libraries commonly used in machine learning like Scikit-learn, TensorFlow, or Keras.

- Begin implementing basic algorithms like linear regression and decision trees on sample datasets provided in the course.

- Set weekly milestones to track progress, such as completing specific modules or implementing models by certain dates.

- Engage with online forums or study groups to discuss concepts and clarify doubts, enhancing understanding through collaboration.

- Start working on the first real dataset for a basic machine learning model, documenting the process and outcomes as part of the project milestones.

- Gradually implement more complex models such as random forests or support vector machines, applying them to different datasets.

- Complete all assessments in the course, aiming for a minimum score of 80%, and review feedback to improve understanding.

- Prepare for the final project by selecting a dataset of interest, defining the problem statement, and outlining the approach and models to be used.

- Conduct the final project, implementing at least three machine learning models, analyzing results, and preparing a presentation to demonstrate findings and insights.

- Review and revise the final project based on feedback from peers or instructors before the presentation.

### 📋 Requirements

- Basic understanding of programming concepts and logic

- Familiarity with Python or R programming language

- Access to a computer with internet for online courses

- Basic knowledge of statistics and mathematics, particularly in algebra and probability

- Ability to use data analysis libraries such as Pandas and NumPy

- Familiarity with data visualization tools like Matplotlib or Seaborn

- Access to a code editor or IDE for programming practice (e.g., Jupyter Notebook, PyCharm)

- Resources for finding reputable online courses (websites like Coursera, edX, or Udacity)

- Time management skills for creating a study schedule and setting milestones

### 📋 Smart Goal

By the end of 6 months, I will complete an online machine learning course that
covers fundamental concepts, algorithms, and practical applications, achieving a
minimum score of 80% on all assessments. I will implement at least 3 basic
machine learning models on real datasets, tracking my progress through weekly
project milestones and a final project presentation to demonstrate my
understanding and ability to analyze data effectively.

In [ ]:
initial_state = {
    "query": "I need to learn the R programming language given I am already proficient in statistics from my major",
    "goal": "",
    "smart_goal": "",
    "curriculum": [],
    "learning_path": [],
    "requirements": [],
}

# Run the graph
result = graph.invoke(initial_state)

2025-07-22 22:02:23,249 - INFO - SQLite cache hit
2025-07-22 22:02:23,271 - INFO - SQLite cache hit
2025-07-22 22:02:23,316 - INFO - SQLite cache hit
2025-07-22 22:02:23,339 - INFO - SQLite cache hit
2025-07-22 22:02:23,375 - INFO - SQLite cache hit


In [ ]:
display_markdown(result, title="R Programming Analysis")

# 🎭 R Programming Analysis

## ❓ Question

> **❝ I need to learn the R programming language given I am already proficient in
statistics from my major ❞**

---

## 📚 Additional Information

### 📋 Curriculum

## 📋 Module 1: Introduction to R Programming

## 📋 Module 2: Data Types and Operators in R

## 📋 Module 3: Control Structures and Functions

## 📋 Module 4: Data Manipulation with dplyr

## 📋 Module 5: Course Completion and Assessment Strategies

## 📋 Module 6: Practical Project 1 - Data Manipulation

## 📋 Module 7: Introduction to Data Visualization with ggplot2

## 📋 Module 8: Statistical Modeling Basics in R

## 📋 Module 9: Course Completion and Final Assessment Strategies

## 📋 Module 10: Practical Project 2 - Statistical Analysis and Visualization

## 📋 Module 11: Advanced R Programming Concepts

## 📋 Module 12: Machine Learning Fundamentals in R

## 📋 Module 13: Synthesis of Knowledge and Course Completion Strategies

## 📋 Module 14: Portfolio Development and Presentation Skills

### 📋 Goal

To acquire proficiency in the R programming language within 3 months by
completing online courses and practical projects, building on my existing
knowledge of statistics.

### 📋 Learning Path

- Research and select 3 online courses focused on R programming that include assessments and practical projects.

- Create a study schedule to allocate time for each course, ensuring you can dedicate at least 5-7 hours per week to learning.

- Begin with the first online course, focusing on fundamental concepts of R programming such as data types, operators, and control structures.

- Complete all course modules and take the quizzes or assessments, aiming for a minimum score of 85%.

- Upon completing the first course, start a practical project that utilizes the concepts learned, such as data manipulation using the dplyr package.

- Complete the first practical project, document your process, and reflect on what you learned.

- Move on to the second online course, diving deeper into data visualization and statistical modeling in R.

- Again, complete all modules and aim for an 85% or higher on the final assessment of the second course.

- After finishing the second course, initiate a second practical project that incorporates statistical analysis and visualization techniques learned from the course.

- Document and showcase your second project, focusing on the insights gathered from your data analysis.

- Finally, enroll in the third online course that covers advanced topics such as machine learning in R or specialized packages relevant to your career goals.

- Complete the third course, ensuring to maintain the 85% benchmark on assessments, and synthesize all knowledge gained.

- Prepare a portfolio including all three course certificates and the two practical projects, highlighting your proficiency in R programming and data analysis skills.

### 📋 Requirements

- Basic understanding of programming concepts (variables, loops, functions)

- Familiarity with statistics and data analysis principles

- Access to a computer with R and RStudio installed

- Basic knowledge of data visualization concepts

- Ability to navigate online learning platforms

- Proficiency in using spreadsheets and data manipulation tools

- Basic understanding of machine learning concepts (for later modules)

- Access to resources like online tutorials, documentation, and forums for R programming

### 📋 Smart Goal

By the end of 3 months, I will achieve proficiency in R programming by
completing 3 online courses with a minimum score of 85% on each course's final
assessments and by successfully completing 2 practical projects that apply my
existing knowledge of statistics, thus enhancing my data analysis skills for my
career advancement.

# Help Request